# 📊 Sales Data Engineering & Analysis Project
This project demonstrates a full data engineering and analysis workflow on a retail sales dataset. It covers data cleaning, transformation, SQL storage, querying, and visualization using **Python, Pandas, NumPy, Matplotlib, Seaborn, and SQLite**.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3


## 📥 Load the Dataset

In [ ]:

df = pd.read_csv('train.csv')

# Convert dates
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True, errors='coerce')

df.head()


## 🧹 Data Cleaning

In [ ]:

# Drop duplicates and handle missing values
df = df.drop_duplicates()
df = df.dropna(subset=['Sales'])

# Add new columns for analysis
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month_name()

df.info()


## 📈 Key Performance Indicators

In [ ]:

total_sales = df['Sales'].sum()
avg_sales = df['Sales'].mean()
top_category = df.groupby('Category')['Sales'].sum().idxmax()
top_state = df.groupby('State')['Sales'].sum().idxmax()

print("Total Sales: $", round(total_sales,2))
print("Average Sales per Order: $", round(avg_sales,2))
print("Top Category:", top_category)
print("Top State:", top_state)


## 📊 Visualizations

In [ ]:

plt.figure(figsize=(10,5))
df.groupby('Year')['Sales'].sum().plot(kind='bar', color='skyblue')
plt.title("Total Sales per Year")
plt.ylabel("Sales")
plt.xlabel("Year")
plt.show()


In [ ]:

plt.figure(figsize=(12,6))
sns.barplot(data=df.groupby('Category')['Sales'].sum().reset_index(),
            x='Category', y='Sales', palette='viridis')
plt.title("Sales by Product Category")
plt.show()


In [ ]:

plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='Category', y='Sales')
plt.title("Sales Distribution by Category")
plt.ylim(0,1000)
plt.show()


In [ ]:

plt.figure(figsize=(12,6))
monthly_sales = df.groupby('Month')['Sales'].sum().reindex([
    'January','February','March','April','May','June',
    'July','August','September','October','November','December'
])
monthly_sales.plot(kind='line', marker='o', color='red')
plt.title("Monthly Sales Trend")
plt.ylabel("Sales")
plt.show()


## 🗄️ Save Data to SQLite and Run Queries

In [ ]:

conn = sqlite3.connect('sales.db')
df.to_sql('sales', conn, if_exists='replace', index=False)

query1 = pd.read_sql("SELECT Category, SUM(Sales) as TotalSales FROM sales GROUP BY Category ORDER BY TotalSales DESC", conn)
query2 = pd.read_sql("SELECT State, SUM(Sales) as TotalSales FROM sales GROUP BY State ORDER BY TotalSales DESC LIMIT 5", conn)
query3 = pd.read_sql("SELECT Year, SUM(Sales) as TotalSales FROM sales GROUP BY Year", conn)

conn.close()

query1, query2, query3
